In [0]:
%pip install --upgrade --force-reinstall /dbfs/tmp/insurance-0.0.1-py3-none-any.whl

In [0]:
%restart_python

In [0]:
import os
import time

import mlflow
import pandas as pd
import requests
from databricks import feature_engineering
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession

from insurance.config import ProjectConfig
from insurance.serving.feature_serving import FeatureServing

In [0]:
config = ProjectConfig.from_yaml(config_path="../project_config.yml")

# COMMAND ----------
spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

fe = feature_engineering.FeatureEngineeringClient()
mlflow.set_registry_uri("databricks-uc")

In [0]:
# get environment variables
os.environ["DBR_TOKEN"] = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
os.environ["DBR_HOST"] = spark.conf.get("spark.databricks.workspaceUrl")

In [0]:
catalog_name = config.catalog_name
schema_name = config.schema_name
feature_table_name = f"{catalog_name}.{schema_name}.insurance_preds"
feature_spec_name = f"{catalog_name}.{schema_name}.return_predictions"
endpoint_name = "insurance-feature-serving"


In [0]:

train_set = spark.table(f"{catalog_name}.{schema_name}.train_set").toPandas()
test_set = spark.table(f"{catalog_name}.{schema_name}.test_set").toPandas()
df = pd.concat([train_set, test_set])

In [0]:

model = mlflow.sklearn.load_model(f"models:/{catalog_name}.{schema_name}.insurance_model_basic@latest-model")

In [0]:
preds_df = df[["Id", "age", "bmi", "children"]]
preds_df["charges"] = model.predict(df[config.cat_features + config.num_features])
preds_df = spark.createDataFrame(preds_df)

In [0]:
fe.create_table(
    name=feature_table_name, primary_keys=["Id"], df=preds_df, description="Insurance predictions feature table"
)

spark.sql(f"""
          ALTER TABLE {feature_table_name}
          SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
        """)

In [0]:
# Initialize feature store manager
feature_serving = FeatureServing(
    feature_table_name=feature_table_name, feature_spec_name=feature_spec_name, endpoint_name=endpoint_name
)

In [0]:
# Create online table
feature_serving.create_online_table()


In [0]:
# Create feature spec
feature_serving.create_feature_spec()

In [0]:
# Deploy feature serving endpoint
feature_serving.deploy_or_update_serving_endpoint()

In [0]:
start_time = time.time()
serving_endpoint = f"https://{os.environ['DBR_HOST']}/serving-endpoints/{endpoint_name}/invocations"
response = requests.post(
    f"{serving_endpoint}",
    headers={"Authorization": f"Bearer {os.environ['DBR_TOKEN']}"},
    json={"dataframe_records": [{"Id": "1094"}]},
)

end_time = time.time()
execution_time = end_time - start_time

print("Response status:", response.status_code)
print("Reponse text:", response.text)
print("Execution time:", execution_time, "seconds")


In [0]:
response = requests.post(
    f"{serving_endpoint}",
    headers={"Authorization": f"Bearer {os.environ['DBR_TOKEN']}"},
    json={"dataframe_split": {"columns": ["Id"], "data": [["1094"]]}},
)